# 第5回 ヒューリスティック関数

## アナウンス

- グラフ探索アルゴリズムIIの方ですが、6/17はすみませんが休講とします。
- 講義の最終回に2022年度授業アンケートの案内をします。是非回答してください。

## 概要

A\*探索は**許容的かつ単調なヒューリスティック関数 (admissible and consistent heuristic)**
がある場合に最善解を返すことはすでに学んだとおり。
また、A\*探索のバリエーションによっては単調性は不要であることも紹介した。
今回はヒューリスティック関数の効果と作り方について説明する。

## ヒューリスティック関数の効果

8パズルは状態数が $9!/2 = 181,400$ 通りしかないので、
全てをメモリ上に置くのも容易だし、全探索で解を見つけることも問題なくできる。
しかし15パズルになると状態数は $15!/2 = 10,461,394,944,000$となって、
これは通常のコンピュータのメモリには載らない。
全探索もかなり困難であり、探索空間をヒューリスティック関数で枝刈りする必要がある。

枝刈りの効果を説明するために **有効分岐因子(effective branching factor)** を
考えることがある。A\*探索が展開した節点数を $N$ として、解が発見された深さを $d$ とする。
この場合、以下の式を満たす $b^*$ を有効分岐因子と言う。
探索空間が、深さが$d$で分岐数が一様に$b$の木とした場合の節点数に相当する。

$ N+1 = 1+b^* + (b^*)^2 + \cdots + (b^*)^d$

当然問題のインスタンスによって$b^*$は異なるが、
多数のインスタンスに対して平均をとれば、
たとえば8パズルの有効分岐因子を測定することができる。
良いヒューリスティック関数は$b^*$を$1$に近づけることで
探索空間が大きな問題を解いていると言える。

実際に8パズルについて、ヒューリスティック関数の影響を測定した結果が以下の図である
（参考文献1, Figure 3.26)。
これは前回も紹介したハミング距離ヒューリスティック ($h_1$) とマンハッタン距離ヒューリスティック ($h_2$) を用いた
A\*探索と幅優先探索 （BFS) の比較をした結果である。
生成された節点数を深さ28まで測定し、それを元に有効分岐因子を計算している。
この結果を見ると BFS よりも $h_1$, $h_2$ を使った方が良く、
また $h_2$ は $h_1$ より性能が良さそうである。

<img src="aima_fig.3.26.png" width=600>

ここで使った$h_1$と$h_2$については、
任意の節点$n$に対して $h_2(n) \ge h_1(n)$という関係がある。
$h_2$が$h_1$に対して **優越する ($h_2$ dominates $h_1$)** という。
A\*探索の場合、$h_2$が$h_1$を超える節点を展開することは基本的にない
（タイブレークの結果で例外的に逆転する可能性はある）。

もう少し詳しく言うと、
最善解のコストを$C^*$とした場合、スコア関数 $f$ に対して
$f(n) < C^*$である節点$n$は全て展開される。
最善解にギリギリ届かない範囲までは全部探索される、とも言える。
$f(n) = g(n) + h(n)$ なので $h$ が
単調なら $h(n) < C^* - g(n)$ である節点$n$ が展開されると言い換えることができる。
$h_2$は常に$h_1$以上だったので$h_2$で展開される節点は$h_1$でも展開される。

以上より、$h$の値は大きいほど良い。
ただし、もちろん許容的であり、また$h$の計算時間があまり大きくならないという条件はある。

## ヒューリスティック関数の作り方

ヒューリスティック関数の質が探索の性能に大きな影響を与えることを紹介した。
ではどのようにして許容的なヒューリスティック関数を作るのか、いくつか考え方を紹介する。

### 緩和問題 (relaxed problem) によるヒューリスティック

単調あるいは許容的な$h$は節点$n$のコストを過大評価してはいけない。
そこで考えられるのが、元の問題を少し簡単にした **緩和問題 (relaxed problem)** を考える方法である。
元の問題の制約を緩めて最善解のコストが必ず元の問題以下になるようにしておいて、
その緩和問題を解けば許容的なヒューリスティック関数を得ることができる。

たとえば8パズルに対するハミング距離ヒューリスティック ($h_1$) はタイルをどこにでも動かして良いという
緩和をしており、
マンハッタン距離ヒューリスティック ($h_2$) は他のタイルと重なっても良いという緩和をしている。

緩和問題は、元の問題のグラフに新たな辺を付け加えたものになる。
元の問題の解は、緩和問題でも（最善ではないが）解である。
緩和問題上の厳密な最適解は元の問題の許容的なヒューリスティックになる。
このヒューリスティックは三角不等式を満たすため、単調でもある。
緩和問題を作ること自体は比較的簡単だが、
ヒューリスティック関数として探索に用いるためには最善解が高速に計算できる必要がある。

ところで当然だが、緩和問題の作り方は多数ありうる。
（自動的に緩和問題を生成する研究もある。）
さきほどの $h_1$, $h_2$ のように優越関係があれば優れた方を選べば良いが、
優越関係のない多数のヒューリスティック関数が得られたとして
それを活用する方法はないだろうか。
例えば $k$ 個の許容的なヒューリスティック関数 $h_1, h_2, h_3, ..., h_k$ があるとする。
この場合、単純に以下の式を使うことができる。

$ h(n) = max \{h_1(n), ... , h_k(n)\} $

元のヒューリスティック関数が全て許容的であるので、この式で得られた $h$ も当然許容的である。
また、 $h$ は元の $h_1, ..., h_k$ の全てに対して優越する。
ヒューリスティック関数の計算に時間がかかることを除けば非常に優れた方法である。
もし計算時間がかかることが問題なら、ランダムに数個選んで max を計算しても良い。
こうすると単調性が失われるが、現実には多くの問題で高速化が可能である。


### 部分問題 (subproblem) によるヒューリスティック (パターンデータベース)

元の問題を **部分問題 (subproblem)** に分割できる場合、
これを利用して許容的なヒューリスティック関数を作ることもできる。
特に **パターンデータベース (pattern database, PDB)** が有効である。
8パズルを例に説明しよう。

<img src="8puzzle.png" width=600>

パターンデータベースの考え方は、元の問題の部分問題について、
その最善解を全列挙して持っておくということである。
たとえば8パズルの場合、1〜4のパネルをそろえる問題は元の問題の部分問題である。
この部分問題の状態空間の大きさは、パネル4枚と空白1枚の5個を考えて
$ 9 x 8 x 7 x 6 x 5 = 15,120$ 通りとなる。
1〜4までを正しい位置に動かすまでの手数を 15,120個のパターンについて調べて
手数を記憶しておけば、パターンデータベースによる一つのヒューリスティック関数として使うことができる。
同様に、5,6,7,8 の4枚についてや、2,4,6,8の4枚についての
パターンデータベースを作ることもできる。
このようにして複数の部分問題に対するパターンデータベースを用意しておき、
それら全ての max をとれば、マンハッタン距離ヒューリスティックよりも正確な
ヒューリスティック関数となる。

8パズルの場合には元が 181,400 通りでこの部分問題が 15,120 通りなので
あまりインパクトがないかもしれない。
しかしもっと大きい問題の場合、うまく部分問題を作ると元の問題よりも
遙かに小さなパターン数のデータベースを作ることができる。

この方法の一つの問題は、複数のヒューリスティック関数の max をとるため、
数を増やすと効果が減衰していくことである。
もし max の代わりに合計することができれば非常に都合が良い。
例えば 1〜4 についてと 5〜8 についての二つのパターンデータベースを用意しておき、
それらを合計したら許容的なヒューリスティック関数になるだろうか？
残念ながらこれはならない。
なぜかというと、1〜4のパネルをそろえている間に5〜8のパネルも都合良く
動くかも知れないからだ。
なので合計してしまうと最善解を超えることがある。

ところが、パターンデータベースを構築する際に簡単な工夫をしておくと、
maxではなく加算できるヒューリスティック関数を作ることができる。
上記の例では、1〜4をそろえるための最善解の手数を求めると説明した。
これには当然 1〜4 以外を動かす手順も含まれている。
そのために加算できないわけだ。
では、1〜4をそろえるための手数に、それ以外のパネルを動かす手数を
含めないことにすればどうだろうか？
この単純な発想でパターンデータベースを構築すると、
加算しても許容的なままであるヒューリスティック関数を作ることができる。
1〜4について、5〜8についてのパターンデータベースによるヒューリスティック $h_1, h_2$ を元に、
節点 $n$ について $h(n) = h_1(n) + h_2(n)$ を許容的なヒューリスティックとすることができる。
これを **disjoint pattern database** と呼ぶ。

たとえば15パズルの場合、8枚と7枚に分割した二つの部分問題についてパターンデータベースを用意すると
マンハッタン距離ヒューリスティックと比較して探索節点数を千分の一以下にできる。
（平均的には探索時間も1秒かからない。）
また、15パズルを一つ大きくした24パズルについても、これを4分割したパターンデータベースを持つことで
現実的な時間で解くことができる。
パターンデータベースは部分問題に分割できる問題であれば非常に有効である。
（詳しくは以下の参考文献を参照。）

> R. E. Korf, A. Felner, "Disjoint pattern database heuristics", Artificial Intelligence, 2002.

### ランドマーク (landmarks) を用いたヒューリスティック

以下の図を見て欲しい。
このような2次元マップ上での最短経路を考える。
例えば左上の付近から右下の付近へ移動するとしよう。
その場合、スタートとゴールが多少ずれても、途中の経路は共通になりそうだ。
最短経路をあらかじめいくつか計算しておき、
それらが共通して通る節点をランドマークとして記憶しておくことで
ヒューリスティック関数を作ることができる。
移動経路を考えるときに、特急の停車駅や高速道路のインターチェンジを手がかりにするようなものと言える。

前処理として色々なスタートやゴールから最短経路探索をしておくことで
自動的にランドマークを得る方法が研究されている。

<img src="dragonage.png" width=600>

### 学習を用いる方法

最近の機械学習の進歩を考えると、
これを利用してヒューリスティック関数を作成することは自然な発想である。
機械学習モデルを訓練して最善解のコストを学習し、その出力をヒューリスティック関数として
使用することが考えられる。

残念ながらこのようなヒューリスティック $h$ は許容的なヒューリスティックとはならないのが普通である。
しかし、$h(n) > h^*(n)$となる確率が十分に低い場合、得られた解と最善解との差は現実には小さいことが多い。
これは非常に有望なテクニックだと思われるが、まだ発展中のテーマである。


### 第5回課題:
- 課題 5-1. 何か適当な探索問題（ただし、2D-gridと8-puzzleは除く)を想定し、それに対するヒューリスティック関数を提示せよ。可能なら許容的又は単調なものを作り、その説明をせよ。
- 課題 5-2. 8パズルについて1〜4のパネルに着目したパターンデータベースを構築せよ (提出するのはコードだけで良いが、パターンデータベース自体を圧縮して添付しても良い)
- 課題 5-3. (発展課題) パターンデータベースを用いて8パズルを解け。加算できる disjoint pattern database によるヒューリスティックでも、maxをとる場合でもどちらでも良い。
- 課題 5-4. (発展課題) パターンデータベースを用いて15パズルを解け。

# 課題1   
探索問題: Rubik's cube problem   
ヒューリスティック関数:   
1) number of colors per plane (admissible, not consistent)   
2) pattern database heuristuc function

# 課題2  
1-4

In [41]:
from pdb import set_trace as st
import copy
import heapq
import itertools

class Node:
    def __init__(self, state, g, h, parent):
        self.state = state
        self.g = g
        self.h = h
        self.parent = parent
        
    def cost(self):
        return self.g + self.h

    def __lt__(self, other):
        if (self.g + self.h) < (other.g + other.h):
            return True
        elif (self.g + self.h) == (other.g + other.h):
            return self.h < other.h
        else:
            return False

class State:
    def __init__(self, pos):
        self.pos = pos

    def __str__(self):
        string = ""
        for i in range(3):
            for j in range(3):
                string += str(self.pos[i][j])
            string += '\n'
        return string

    def __eq__(self, other):
        return self.pos == other.pos

    def get_position(self, num):
        for i in range(3):
            for j in range(3):
                if self.pos[i][j] == num:
                    return (i, j)
        # should throw exception
        return None

    def expand(self):
        children=[]
        x,y=self.get_position(0)
        if x==0:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x+1][y]=state[x+1][y],state[x][y]
            children.append(State(state))
        if y==0:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y+1]=state[x][y+1],state[x][y]
            children.append(State(state))
        if x==1:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x+1][y]=state[x+1][y],state[x][y]
            children.append(State(state))
            state=copy.deepcopy(self.pos)
            state[x][y],state[x-1][y]=state[x-1][y],state[x][y]
            children.append(State(state))
        if y==1:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y+1]=state[x][y+1],state[x][y]
            children.append(State(state))
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y-1]=state[x][y-1],state[x][y]
            children.append(State(state))
        if x==2:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x-1][y]=state[x-1][y],state[x][y]
            children.append(State(state))
        if y==2:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y-1]=state[x][y-1],state[x][y]
            children.append(State(state))
        return children


class Puzzle:
    def __init__(self, i, num):
        if num==0:
            self.start=State(i)
            self.goal = State([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
        elif num==1:
            st=[i[:3],[i[3],i[4], 5],[7, 8, 6]]
            self.start = State(st)
            self.goal = State([[1, 2, 3], [4, 0, 5],[7, 8, 6]])
        elif num==5:
            st=[[1,2,3],[4,0,i[0]],[i[1],i[2],i[3]]]
            self.start = State(st)
            self.goal = State([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

    def heuristic(self, state):
        # manhattan
        h_value=0
        for i in range(5,9):
            x,y=state.get_position(i)
            x1,y1=self.goal.get_position(i)
            h_value+=abs(x-x1)+abs(y-y1)
        return h_value
                
    def hamming(self,state):
        h_value=0
        for i in range(9):
            x,y=state.get_position(i)
            x1,y1=self.goal.get_position(i)
            if x!=x1 or y!=y1:
                h_value+=1
        return h_value

class AStar:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.closed_list = { }

    def solution_path(self, goal):
        solution = []
        node = goal
        while True:
            solution = [str(node.state)] + solution
            if node.parent.pos[0][0] < 0:
                break
            node = self.closed_list[str(node.parent)]
        sol_str = "len: " + str(goal.g) + '\n'
        for pos in solution:
            sol_str += pos + '\n'
        return sol_str

    def search(self):
        start_state = self.puzzle.start
        dummy_state = State([[-1, -1, -1], [-1, -1, -1], [-1, -1, -1]])
        
        # manhattan
        start_node = Node(start_state, 0, self.puzzle.heuristic(start_state), dummy_state)
        
        # # hamming
        # start_node = Node(start_state, 0, self.puzzle.hamming(start_state), dummy_state)

        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        self.closed_list = { }
        self.closed_list[str(start_state)] = start_node

        if start_state == self.puzzle.goal:
            return self.solution_path(start_node)
        
        count_expand=0
        while len(open_queue) != 0 and count_expand<300:
            n = heapq.heappop(open_queue)
            if n.state == self.puzzle.goal:
                return n
            child = n.state.expand()
            count_expand+=1
            for c in child:
                num=self.puzzle.heuristic(c)
                # num=self.puzzle.hamming(c)
                num1=n.g
                if str(c) not in self.closed_list or (num1+1+num) < self.closed_list[str(c)].cost():
                    newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                    # newnode=Node(c,num1+1,self.puzzle.hamming(c),n)
                    self.closed_list[str(c)] = newnode
                    heapq.heappush(open_queue, newnode)
        return None
    
    
    def solve_by_pattern14(self,pattern14):
        start_state = self.puzzle.start
        dummy_state = State([[-1, -1, -1], [-1, -1, -1], [-1, -1, -1]])
        
        # manhattan
        start_node = Node(start_state, 0, self.puzzle.heuristic(start_state), dummy_state)
        
        # # hamming
        # start_node = Node(start_state, 0, self.puzzle.hamming(start_state), dummy_state)

        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        self.closed_list = { }
        self.closed_list[str(start_state)] = start_node

        if start_state == self.puzzle.goal:
            return self.solution_path(start_node)
        
        count_expand=0
        while len(open_queue) != 0 and count_expand<1000:
            n = heapq.heappop(open_queue)
            st_14=str(n.state.pos[0][0])+str(n.state.pos[0][1])+str(n.state.pos[0][2])+str(n.state.pos[1][0])+str(n.state.pos[1][1])
            if st_14 in pattern14:
                return n
            child = n.state.expand()
            count_expand+=1
            for c in child:
                num=self.puzzle.heuristic(c)
                # num=self.puzzle.hamming(c)
                num1=n.g
                if str(c) not in self.closed_list or (num1+1+num) < self.closed_list[str(c)].cost():
                    newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                    # newnode=Node(c,num1+1,self.puzzle.hamming(c),n)
                    self.closed_list[str(c)] = newnode
                    heapq.heappush(open_queue, newnode)
        return None
    
def pattern_14():
    s=[0,1,2,3,4]
    genr=itertools.permutations(s,5)
    a=next(genr)
    pattern_14={}
    try:
        while a:
            str1=''
            for i in a:
                str1+=str(i)
            pz4=Puzzle(list(a),num=1)
            solver = AStar(pz4)
            solution = solver.search()
            a8=solution
            if a8:
                pattern_14[str1]=a8
            a=next(genr)
    except StopIteration:
        return pattern_14

    
pattern14=pattern_14()
print(pattern14)

{'01342': <__main__.Node object at 0x00000231760FAE50>, '02314': <__main__.Node object at 0x00000231760CA340>, '03412': <__main__.Node object at 0x0000023175FFEF40>, '04132': <__main__.Node object at 0x00000231760D3AC0>, '04321': <__main__.Node object at 0x0000023176101250>, '10342': <__main__.Node object at 0x000002317600F880>, '12034': <__main__.Node object at 0x0000023175F7C2E0>, '12304': <__main__.Node object at 0x00000231760FACA0>, '12340': 'len: 0\n123\n405\n786\n\n', '13042': <__main__.Node object at 0x00000231760FAEE0>, '20314': <__main__.Node object at 0x000002317600F0A0>, '23014': <__main__.Node object at 0x0000023176101310>, '24301': <__main__.Node object at 0x000002317610D430>, '24310': <__main__.Node object at 0x000002317610D610>, '30412': <__main__.Node object at 0x0000023176038070>, '31024': <__main__.Node object at 0x000002317537E400>, '32410': <__main__.Node object at 0x000002317600E5B0>, '34012': <__main__.Node object at 0x000002317600E4F0>, '40132': <__main__.Node ob

In [43]:
a8=pattern14.get('43021')
num=-1
while hasattr(a8,'parent'):
    print(str(a8.state))
    a8=a8.parent

123
405
786

103
425
786

013
425
786

413
025
786

413
205
786

403
215
786

430
215
786



# 課題3

In [29]:
def pattern_58():
    s=[5,6,7,8]
    genr=itertools.permutations(s,4)
    a=next(genr)
    pattern_58={}
    try:
        while a:
            str1=''
            for i in a:
                str1+=str(i)
            pz4=Puzzle(list(a),num=5)
            solver = AStar(pz4)
            solution = solver.search()
            a8=solution
            if a8:
                pattern_58[str1]=a8
            a=next(genr)
    except StopIteration:
        return pattern_58

pattern58=pattern_58()
print(pattern58)

{'5678': <__main__.Node object at 0x00000231760CAD60>, '5786': <__main__.Node object at 0x000002317600E850>, '5867': <__main__.Node object at 0x000002317610B9D0>, '6758': <__main__.Node object at 0x00000231760548B0>, '7685': <__main__.Node object at 0x00000231760C74C0>, '7856': <__main__.Node object at 0x0000023175FF2FA0>, '8657': <__main__.Node object at 0x0000023175FFCDC0>, '8765': <__main__.Node object at 0x00000231760D0760>}


In [44]:
pzz8=Puzzle([[4,2,3],[7,8,0],[5,1,6]],num=0)
# pzz8=Puzzle([[1,2,3],[4,5,6],[0,7,8]],num=0)
solver = AStar(pzz8)
solution = solver.solve_by_pattern14(pattern14)
ap=solution
if ap:
    while hasattr(ap,'parent'):
        print(str(ap.state))
        ap=ap.parent
else:
    print('no answer')

430
216
758

436
210
758

436
201
758

436
021
758

436
721
058

436
721
508

436
721
580

436
720
581

430
726
581

403
726
581

423
706
581

423
786
501

423
786
510

423
780
516



In [47]:
# 43012abcd --> 12340abcd (abcd=6758)
a_14=pattern14.get('43021')
print(a_14)
while hasattr(a_14,'parent'):
    print(str(a_14.state))
    a_14=a_14.parent

123
405
786

103
425
786

013
425
786

413
025
786

413
205
786

403
215
786

430
215
786



In [48]:
# 123406758 --> 123456780
a_58=pattern58.get('6758')
print(a_58)
while hasattr(a_58,'parent'):
    print(str(a_58.state))
    a_58=a_58.parent

123
456
780

123
456
708

123
406
758



# 課題4  
15-puzzles

In [197]:
from pdb import set_trace as st
import copy
import heapq

class Node:
    def __init__(self, state, g, h, parent):
        self.state = state
        self.g = g
        self.h = h
        self.parent = parent
        
    def cost(self):
        return self.g + self.h

    def __lt__(self, other):
        if (self.g + self.h) < (other.g + other.h):
            return True
        elif (self.g + self.h) == (other.g + other.h):
            return self.h < other.h
        else:
            return False

class State:
    def __init__(self, pos):
        '''
        state: panel position in 3x3 integer list
        '''
        self.pos = pos

    def __str__(self):
        string = ""
        for i in range(4):
            for j in range(4):
                string += str(self.pos[i][j]).zfill(2)+' '
            string += '\n'
        return string

    def __eq__(self, other):
        return self.pos == other.pos

    def get_position(self, num):
        for i in range(4):
            for j in range(4):
                if self.pos[i][j] == num:
                    return (i, j)
        # should throw exception
        return None

    def expand(self):
        '''
        return children state list
        この関数を実装せよ
        '''
        children=[]
        x,y=self.get_position(0)
        if x==0:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x+1][y]=state[x+1][y],state[x][y]
            children.append(State(state))
        if y==0:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y+1]=state[x][y+1],state[x][y]
            children.append(State(state))
        if x==1 or x==2:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x+1][y]=state[x+1][y],state[x][y]
            children.append(State(state))
            state=copy.deepcopy(self.pos)
            state[x][y],state[x-1][y]=state[x-1][y],state[x][y]
            children.append(State(state))
        if y==1 or y==2:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y+1]=state[x][y+1],state[x][y]
            children.append(State(state))
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y-1]=state[x][y-1],state[x][y]
            children.append(State(state))
        if x==3:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x-1][y]=state[x-1][y],state[x][y]
            children.append(State(state))
        if y==3:
            state=copy.deepcopy(self.pos)
            state[x][y],state[x][y-1]=state[x][y-1],state[x][y]
            children.append(State(state))
        return children


class Puzzle:
    def __init__(self, i, num):
        if num==0:
            self.start=State(i)
            self.goal = State([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]])
        elif num==1:
            st=[i[:4],[5,i[4],6,7],[9, 10, 11, 8],[13, 14, 15, 12]]
            self.start = State(st)
            self.goal = State([[1, 2, 3, 4], [5, 0, 6, 7], [9, 10, 11, 8], [13, 14, 15, 12]])
        elif num==2:
            st=[[1,2,3,4],[i[0],0,6,7],[i[1],i[2],11,8],[i[3],i[4],15,12]]
            self.start = State(st)
            self.goal = State([[1, 2, 3, 4], [5, 0, 6, 7], [9, 10, 11, 8], [13, 14, 15, 12]])
        elif num==3:
            st=[[1,2,3,4],[5,0,i[0],i[1]],[9,10,i[2],i[3]],[13,14,i[4],i[5]]]
            self.start = State(st)
            self.goal = State([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]])

    def heuristic(self, state):
        h_value=0
        for i in range(16):
            x,y=state.get_position(i)
            x1,y1=self.goal.get_position(i)
            h_value+=abs(x-x1)+abs(y-y1)
        return h_value
                
class AStar:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.closed_list = { }

    def solution_path(self, goal):
        solution = []
        node = goal
        while True:
            solution = [str(node.state)] + solution
            if node.parent.pos[0][0] < 0:
                break
            node = self.closed_list[str(node.parent)]
        sol_str = "len: " + str(goal.g) + '\n'
        for pos in solution:
            sol_str += pos + '\n'
        return sol_str

    def search(self):
        start_state = self.puzzle.start
        dummy_state = State([[-1, -1, -1,-1], [-1,-1, -1, -1], [-1,-1, -1, -1],[-1,-1,-1,-1]])
        start_node = Node(start_state, 0, self.puzzle.heuristic(start_state), dummy_state)
        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        self.closed_list = { }
        self.closed_list[str(start_state)] = start_node

        if start_state == self.puzzle.goal:
            return self.solution_path(start_node)
        
        count_expand=0
        while len(open_queue) != 0 and count_expand < 300:
            n = heapq.heappop(open_queue)
            if n.state == self.puzzle.goal:
                return n
            child = n.state.expand()
            count_expand+=1
            for c in child:
                num=self.puzzle.heuristic(c)
                num1=n.g
                if str(c) not in self.closed_list or (num1+1+num) < self.closed_list[str(c)].cost():
                    newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                    self.closed_list[str(c)] = newnode
                    heapq.heappush(open_queue, newnode)
        return None
    
    def search_by_pattern(self,pattern1,pattern2,pattern3,numx):
        start_state = self.puzzle.start
        dummy_state = State([[-1, -1, -1,-1], [-1,-1, -1, -1], [-1,-1, -1, -1],[-1,-1,-1,-1]])
        start_node = Node(start_state, 0, self.puzzle.heuristic(start_state), dummy_state)
        open_queue = [ start_node ]
        heapq.heapify(open_queue)
        self.closed_list = { }
        self.closed_list[str(start_state)] = start_node

        if start_state == self.puzzle.goal:
            return self.solution_path(start_node)
        
        count_expand=0
        while len(open_queue) != 0 and count_expand < 50000:
            n = heapq.heappop(open_queue)
            st_1=str(n.state.pos[0][0])+str(n.state.pos[0][1])+str(n.state.pos[0][2])+str(n.state.pos[0][3])+str(n.state.pos[1][1])
            st_2=str(n.state.pos[1][0])+str(n.state.pos[2][0])+str(n.state.pos[2][1])+str(n.state.pos[3][0])+str(n.state.pos[3][1])
            st_3=str(n.state.pos[1][2])+str(n.state.pos[1][3])+str(n.state.pos[2][2])+str(n.state.pos[2][3])+str(n.state.pos[3][2])+str(n.state.pos[3][3])
            if numx==1 and st_1 in pattern1 :
                return n 
            elif numx==2 and st_2 in pattern2:
                return n 
            elif numx==3 and st_3 in pattern3 :
                return n 
            child = n.state.expand()
            count_expand+=1
            for c in child:
                num=self.puzzle.heuristic(c)
                num1=n.g
                if str(c) not in self.closed_list or (num1+1+num) < self.closed_list[str(c)].cost():
                    newnode=Node(c,num1+1,self.puzzle.heuristic(c),n)
                    self.closed_list[str(c)] = newnode
                    heapq.heappush(open_queue, newnode)
        return None
    

In [85]:
def pattern_1():
    s=[0,1,2,3,4]
    genr=itertools.permutations(s,5)
    a=next(genr)
    pattern_1={}
    try:
        while a:
            str1=''
            for i in a:
                str1+=str(i)
            pz4=Puzzle(list(a),num=1)
            solver = AStar(pz4)
            solution = solver.search()
            a15=solution
            if a15:
                pattern_1[str1]=a15
            a=next(genr)
    except StopIteration:
        return pattern_1

    
pattern1=pattern_1()
print(pattern1)

{'01234': <__main__.Node object at 0x0000023176AC33A0>, '01342': <__main__.Node object at 0x0000023176BAE160>, '01423': <__main__.Node object at 0x00000231767E9490>, '02143': <__main__.Node object at 0x0000023176901730>, '10234': <__main__.Node object at 0x0000023176A41520>, '10342': <__main__.Node object at 0x0000023176A412B0>, '10423': <__main__.Node object at 0x00000231768A8070>, '12034': <__main__.Node object at 0x0000023176A4E940>, '12304': <__main__.Node object at 0x0000023176A552B0>, '12340': 'len: 0\n01 02 03 04 \n05 00 06 07 \n09 10 11 08 \n13 14 15 12 \n\n', '13042': <__main__.Node object at 0x00000231768DE3A0>, '13402': <__main__.Node object at 0x00000231768765B0>, '13420': <__main__.Node object at 0x0000023176D059A0>, '14023': <__main__.Node object at 0x00000231768D0730>, '14203': <__main__.Node object at 0x00000231767C8280>, '14230': <__main__.Node object at 0x00000231767E6850>, '20143': <__main__.Node object at 0x0000023176A41EE0>, '30241': <__main__.Node object at 0x0000

In [87]:
a_1=pattern1.get('34102')
print(a_1)
while hasattr(a_1,'parent'):
    print(str(a_1.state))
    a_1=a_1.parent

01 02 03 04 
05 00 06 07 
09 10 11 08 
13 14 15 12 

01 00 03 04 
05 02 06 07 
09 10 11 08 
13 14 15 12 

00 01 03 04 
05 02 06 07 
09 10 11 08 
13 14 15 12 

05 01 03 04 
00 02 06 07 
09 10 11 08 
13 14 15 12 

05 01 03 04 
02 00 06 07 
09 10 11 08 
13 14 15 12 

05 00 03 04 
02 01 06 07 
09 10 11 08 
13 14 15 12 

05 03 00 04 
02 01 06 07 
09 10 11 08 
13 14 15 12 

05 03 04 00 
02 01 06 07 
09 10 11 08 
13 14 15 12 

05 03 04 07 
02 01 06 00 
09 10 11 08 
13 14 15 12 

05 03 04 07 
02 01 00 06 
09 10 11 08 
13 14 15 12 

05 03 04 07 
02 00 01 06 
09 10 11 08 
13 14 15 12 

05 03 04 07 
00 02 01 06 
09 10 11 08 
13 14 15 12 

00 03 04 07 
05 02 01 06 
09 10 11 08 
13 14 15 12 

03 00 04 07 
05 02 01 06 
09 10 11 08 
13 14 15 12 

03 04 00 07 
05 02 01 06 
09 10 11 08 
13 14 15 12 

03 04 01 07 
05 02 00 06 
09 10 11 08 
13 14 15 12 

03 04 01 07 
05 02 06 00 
09 10 11 08 
13 14 15 12 

03 04 01 00 
05 02 06 07 
09 10 11 08 
13 14 15 12 



In [91]:
def pattern_2():
    s=[5,9,10,13,14]
    genr=itertools.permutations(s,5)
    a=next(genr)
    pattern_2={}
    try:
        while a:
            str1=''
            for i in a:
                str1+=str(i)
            pz4=Puzzle(list(a),num=2)
            solver = AStar(pz4)
            solution = solver.search()
            a15=solution
            if a15:
                pattern_2[str1]=a15
            a=next(genr)
    except StopIteration:
        return pattern_2

    
pattern2=pattern_2()
print(pattern2)

{'59101314': 'len: 0\n01 02 03 04 \n05 00 06 07 \n09 10 11 08 \n13 14 15 12 \n\n', '59131410': <__main__.Node object at 0x0000023176EFE880>, '59141013': <__main__.Node object at 0x0000023176EA8910>, '51013914': <__main__.Node object at 0x0000023176EE3A60>, '51391014': <__main__.Node object at 0x0000023176EE4FD0>, '51310149': <__main__.Node object at 0x0000023176962520>, '51314910': <__main__.Node object at 0x0000023176F10160>, '51410913': <__main__.Node object at 0x0000023176F2ED90>, '51413109': <__main__.Node object at 0x0000023176F10B50>, '91051314': <__main__.Node object at 0x0000023176F34C40>, '91013145': <__main__.Node object at 0x0000023176F08C40>, '91014513': <__main__.Node object at 0x0000023176901910>, '91351410': <__main__.Node object at 0x0000023176F34310>, '91310514': <__main__.Node object at 0x0000023176989220>, '91314105': <__main__.Node object at 0x0000023176EF94F0>, '91451013': <__main__.Node object at 0x0000023176F0FEE0>, '91413510': <__main__.Node object at 0x00000231

In [92]:
a_2=pattern2.get('14135109')
print(a_2)
while hasattr(a_2,'parent'):
    print(str(a_2.state))
    a_2=a_2.parent

01 02 03 04 
05 00 06 07 
09 10 11 08 
13 14 15 12 

01 02 03 04 
05 10 06 07 
09 00 11 08 
13 14 15 12 

01 02 03 04 
05 10 06 07 
09 14 11 08 
13 00 15 12 

01 02 03 04 
05 10 06 07 
09 14 11 08 
00 13 15 12 

01 02 03 04 
05 10 06 07 
00 14 11 08 
09 13 15 12 

01 02 03 04 
05 10 06 07 
14 00 11 08 
09 13 15 12 

01 02 03 04 
05 00 06 07 
14 10 11 08 
09 13 15 12 

01 02 03 04 
00 05 06 07 
14 10 11 08 
09 13 15 12 

01 02 03 04 
14 05 06 07 
00 10 11 08 
09 13 15 12 

01 02 03 04 
14 05 06 07 
10 00 11 08 
09 13 15 12 

01 02 03 04 
14 05 06 07 
10 13 11 08 
09 00 15 12 

01 02 03 04 
14 05 06 07 
10 13 11 08 
00 09 15 12 

01 02 03 04 
14 05 06 07 
00 13 11 08 
10 09 15 12 

01 02 03 04 
14 05 06 07 
13 00 11 08 
10 09 15 12 

01 02 03 04 
14 00 06 07 
13 05 11 08 
10 09 15 12 



In [94]:
def pattern_3():
    s=[6,7,11,8,15,12]
    genr=itertools.permutations(s,6)
    a=next(genr)
    pattern_2={}
    try:
        while a:
            str1=''
            for i in a:
                str1+=str(i)
            pz4=Puzzle(list(a),num=3)
            solver = AStar(pz4)
            solution = solver.search()
            a15=solution
            if a15:
                pattern_2[str1]=a15
            a=next(genr)
    except StopIteration:
        return pattern_2

    
pattern3=pattern_3()
print(pattern3)

{'671181512': <__main__.Node object at 0x0000023176EDB8B0>, '671115128': <__main__.Node object at 0x0000023176F375E0>, '671112815': <__main__.Node object at 0x0000023176D19CA0>, '678121511': <__main__.Node object at 0x0000023176F38A30>, '671511812': <__main__.Node object at 0x0000023176EF6520>, '671581211': <__main__.Node object at 0x0000023176EE0700>, '671512118': <__main__.Node object at 0x0000023176EE4220>, '671211158': <__main__.Node object at 0x0000023176EDF0A0>, '671281115': <__main__.Node object at 0x00000231767A9580>, '671215811': <__main__.Node object at 0x0000023176EC1B80>, '611871512': <__main__.Node object at 0x0000023176EE4910>, '611815127': <__main__.Node object at 0x0000023176E9DE50>, '611812715': <__main__.Node object at 0x0000023176ED7250>, '611157128': <__main__.Node object at 0x0000023176D13340>, '611151287': <__main__.Node object at 0x0000023176EB42B0>, '611127815': <__main__.Node object at 0x0000023176D38580>, '611128157': <__main__.Node object at 0x0000023176E9028

In [95]:
a_3=pattern3.get('156781211')
print(a_3)
while hasattr(a_3,'parent'):
    print(str(a_3.state))
    a_3=a_3.parent

01 02 03 04 
05 06 07 08 
09 10 11 12 
13 14 15 00 

01 02 03 04 
05 06 07 08 
09 10 11 00 
13 14 15 12 

01 02 03 04 
05 06 07 08 
09 10 00 11 
13 14 15 12 

01 02 03 04 
05 06 00 08 
09 10 07 11 
13 14 15 12 

01 02 03 04 
05 00 06 08 
09 10 07 11 
13 14 15 12 

01 02 03 04 
05 10 06 08 
09 00 07 11 
13 14 15 12 

01 02 03 04 
05 10 06 08 
09 07 00 11 
13 14 15 12 

01 02 03 04 
05 10 06 08 
09 07 15 11 
13 14 00 12 

01 02 03 04 
05 10 06 08 
09 07 15 11 
13 14 12 00 

01 02 03 04 
05 10 06 08 
09 07 15 00 
13 14 12 11 

01 02 03 04 
05 10 06 00 
09 07 15 08 
13 14 12 11 

01 02 03 04 
05 10 00 06 
09 07 15 08 
13 14 12 11 

01 02 03 04 
05 10 15 06 
09 07 00 08 
13 14 12 11 

01 02 03 04 
05 10 15 06 
09 00 07 08 
13 14 12 11 

01 02 03 04 
05 00 15 06 
09 10 07 08 
13 14 12 11 



In [193]:
# pattern1
# 01 02 03 04
# xx 00 xx xx
# xx xx xx xx 
# xx xx xx xx
pzz15=Puzzle([[10,1,15,0],[7,4,3,2],[11,9,13,8],[6,12,14,5]],num=0)
#pzz15 = Puzzle([[1,3,0,4],[5,2,7,8],[9,6,10,11],[13,14,15,12]],num=0)
solver = AStar(pzz15)
solution = solver.search_by_pattern(pattern1,pattern2,pattern3,numx=1)
ap=solution
if ap:
    while hasattr(ap,'parent'):
        print(str(ap.state))
        ap=ap.parent
else:
    print('no answer')

01 02 03 04 
10 00 15 07 
11 09 13 08 
06 12 14 05 

01 02 03 04 
10 15 00 07 
11 09 13 08 
06 12 14 05 

01 02 03 04 
10 15 07 00 
11 09 13 08 
06 12 14 05 

01 02 03 00 
10 15 07 04 
11 09 13 08 
06 12 14 05 

01 02 00 03 
10 15 07 04 
11 09 13 08 
06 12 14 05 

01 00 02 03 
10 15 07 04 
11 09 13 08 
06 12 14 05 

01 15 02 03 
10 00 07 04 
11 09 13 08 
06 12 14 05 

01 15 02 03 
10 07 00 04 
11 09 13 08 
06 12 14 05 

01 15 02 03 
10 07 04 00 
11 09 13 08 
06 12 14 05 

01 15 02 00 
10 07 04 03 
11 09 13 08 
06 12 14 05 

01 15 00 02 
10 07 04 03 
11 09 13 08 
06 12 14 05 

01 00 15 02 
10 07 04 03 
11 09 13 08 
06 12 14 05 

00 01 15 02 
10 07 04 03 
11 09 13 08 
06 12 14 05 

10 01 15 02 
00 07 04 03 
11 09 13 08 
06 12 14 05 

10 01 15 02 
07 00 04 03 
11 09 13 08 
06 12 14 05 

10 01 15 02 
07 04 00 03 
11 09 13 08 
06 12 14 05 

10 01 15 02 
07 04 03 00 
11 09 13 08 
06 12 14 05 

10 01 15 00 
07 04 03 02 
11 09 13 08 
06 12 14 05 



In [194]:
pzz15=Puzzle([[1,2,3,4],[10,0,15,7],[11,9,13,8],[6,12,14,5]],num=0)
solver = AStar(pzz15)
solution = solver.search_by_pattern(pattern1,pattern2,pattern3,numx=3)
ap=solution
if ap:
    while hasattr(ap,'parent'):
        print(str(ap.state))
        ap=ap.parent
else:
    print('no answer')

01 02 03 04 
09 00 06 07 
10 05 15 12 
13 14 11 08 

01 02 03 04 
09 06 00 07 
10 05 15 12 
13 14 11 08 

01 02 03 04 
09 06 15 07 
10 05 00 12 
13 14 11 08 

01 02 03 04 
09 06 15 07 
10 00 05 12 
13 14 11 08 

01 02 03 04 
09 06 15 07 
10 14 05 12 
13 00 11 08 

01 02 03 04 
09 06 15 07 
10 14 05 12 
13 11 00 08 

01 02 03 04 
09 06 15 07 
10 14 00 12 
13 11 05 08 

01 02 03 04 
09 06 15 07 
10 14 12 00 
13 11 05 08 

01 02 03 04 
09 06 15 07 
10 14 12 08 
13 11 05 00 

01 02 03 04 
09 06 15 07 
10 14 12 08 
13 11 00 05 

01 02 03 04 
09 06 15 07 
10 14 00 08 
13 11 12 05 

01 02 03 04 
09 06 15 07 
10 00 14 08 
13 11 12 05 

01 02 03 04 
09 00 15 07 
10 06 14 08 
13 11 12 05 

01 02 03 04 
00 09 15 07 
10 06 14 08 
13 11 12 05 

01 02 03 04 
10 09 15 07 
00 06 14 08 
13 11 12 05 

01 02 03 04 
10 09 15 07 
06 00 14 08 
13 11 12 05 

01 02 03 04 
10 09 15 07 
06 11 14 08 
13 00 12 05 

01 02 03 04 
10 09 15 07 
06 11 14 08 
00 13 12 05 

01 02 03 04 
10 09 15 07 
00 11 14 08 
06 13 1

In [183]:
# pattern2
# xx xx xx xx
# 05 00 xx xx
# 09 10 xx xx 
# 13 14 xx xx
a_2=pattern2.get('91051314')
print(a_2)
while hasattr(a_2,'parent'):
    print(str(a_2.state))
    a_2=a_2.parent

01 02 03 04 
05 00 06 07 
09 10 11 08 
13 14 15 12 

01 02 03 04 
00 05 06 07 
09 10 11 08 
13 14 15 12 

01 02 03 04 
09 05 06 07 
00 10 11 08 
13 14 15 12 

01 02 03 04 
09 05 06 07 
10 00 11 08 
13 14 15 12 

01 02 03 04 
09 00 06 07 
10 05 11 08 
13 14 15 12 



In [182]:
# pattern3
# 01 02 03 04   ----->   01 02 03 04  ----->   01 02 03 04
# xx 00 aa bb   ----->   xx 00 06 07  ----->   xx 06 07 08
# xx xx cc dd   ----->   xx xx 11 08  ----->   xx xx 11 12
# xx xx ee ff   ----->   xx xx 15 12  ----->   xx xx 15 00
a_3=pattern3.get('671512118')
print(a_3)
while hasattr(a_3,'parent'):
    print(str(a_3.state))
    a_3=a_3.parent

01 02 03 04 
05 06 07 08 
09 10 11 12 
13 14 15 00 

01 02 03 04 
05 06 07 08 
09 10 11 00 
13 14 15 12 

01 02 03 04 
05 06 07 08 
09 10 00 11 
13 14 15 12 

01 02 03 04 
05 06 07 08 
09 10 15 11 
13 14 00 12 

01 02 03 04 
05 06 07 08 
09 10 15 11 
13 14 12 00 

01 02 03 04 
05 06 07 08 
09 10 15 00 
13 14 12 11 

01 02 03 04 
05 06 07 00 
09 10 15 08 
13 14 12 11 

01 02 03 04 
05 06 00 07 
09 10 15 08 
13 14 12 11 

01 02 03 04 
05 06 15 07 
09 10 00 08 
13 14 12 11 

01 02 03 04 
05 06 15 07 
09 10 12 08 
13 14 00 11 

01 02 03 04 
05 06 15 07 
09 10 12 08 
13 14 11 00 

01 02 03 04 
05 06 15 07 
09 10 12 00 
13 14 11 08 

01 02 03 04 
05 06 15 07 
09 10 00 12 
13 14 11 08 

01 02 03 04 
05 06 00 07 
09 10 15 12 
13 14 11 08 

01 02 03 04 
05 00 06 07 
09 10 15 12 
13 14 11 08 



参考文献
1. "Artificial Intelligence: A Modern Approach, 4th Global ed.", by Stuart Russell and Peter Norvig  
   http://aima.cs.berkeley.edu/index.html
1. "ヒューリスティック探索入門", 陣内 佑  
   https://jinnaiyuu.github.io/pdf/textbook.pdf